In [66]:
#Install tabula-py package
!pip install tabula-py
#Import Libraries
import numpy as np
import pandas as pd
from sklearn import linear_model
import sklearn
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
from google.colab import drive
import os
import pandas as pd
import tabula as tabula

# @title Please fill in the loan aplication entries
Loan_ID = 11123 # @param {type:"integer"}
Gender = 2 # @param ["1", "2"] {type:"raw"}
Married = 1 # @param ["0", "1"] {type:"raw"}
Dependents = 3 # @param {type:"integer"}
Graduate = 1 # @param ["0", "1"] {type:"raw"}
Self_Employed = 1 # @param ["0", "1"] {type:"raw"}
ApplicantIncome = 3000 # @param {type:"number"}
CoapplicantIncome = 1600 # @param {type:"number"}
LoanAmount = 100 # @param {type:"number"}
Loan_Amount_Term = 300 # @param {type:"number"}
Credit_History = 1 # @param ["0", "1"] {type:"raw"}
Property_Area = 2 # @param ["1", "2", "3"] {type:"raw"}

#Parameters for the loan testing model
Model_Test_Size = 0.1 # @param {type:"number"}
Model_Iterations = 10000 # @param {type:"number"}

#Create an array to save potential loan client data
clientData = np.array([Gender, Married, Dependents, Graduate, Self_Employed, ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term, Credit_History, Property_Area])
clientData = clientData.reshape(1,11)

# Mount Drive
drive.mount('/content/gdrive')

# Change the current working directory to the directory containing the file you want to open
os.chdir('/content/gdrive/My Drive/Colab Notebooks')

# Read the Zappy Loan Data.xlsx file
data1 = pd.read_excel('Zappy Loan Data.xlsx')

# Read the Loans_Database_Table.pdf file into a list of pandas DataFrames
tables = tabula.read_pdf("Loans_Database_Table.pdf", pages = 'all')

"""Check whether a appendedDataFrames.csv file exsists and
add an index to an already existing one to create a
new one, where we will append all the pdf dataframes
from the tables """

def create_unique_file(filename):
    base_name, file_extension = os.path.splitext(filename)
    index = 1

    while True:
        new_filename = f"{base_name}_{index}{file_extension}"

        if not os.path.exists(new_filename):
            with open(new_filename, 'w') as file:
                pass
            return new_filename

        index += 1

filename = "appendedPdfDataFrames.csv"
new_file = create_unique_file(filename)

# Append all the dataframes from all the tables to a csv file called appendedPdfDataFrames.csv
def append_dataframes_to_csv(df_list, filename, mode='a'):

    column_names = ["Loan_ID","Gender", "Married", "Dependents","Graduate", "Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History","Property_Area","Loan_Status"]

    with open(filename, mode) as f:
        if mode == 'w':
            f.write(','.join(column_names) + '\n')
        for df in df_list:
            df.to_csv(f, header=False, index=False)


append_dataframes_to_csv([tables[0],tables[1],tables[2],tables[3],tables[4]
                          ,tables[5],tables[6],tables[7],tables[8],tables[9]
                          ,tables[10],tables[11],tables[12],tables[13]], new_file, mode='a')

#Add column names
def add_column_names_to_csv(filename, column_names):
    with open(filename, 'r') as f:
        data = f.readlines()

    with open(filename, 'w') as f:
        f.write(','.join(column_names) + '\n')
        for line in data:
            f.write(line)



column_names = ["Loan_ID","Gender", "Married", "Dependents","Graduate", "Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History","Property_Area","Loan_Status"]


add_column_names_to_csv(new_file, column_names)
# Read the appendedPdfDataFrames.csv file and store the data in data2 variable
data2 = pd.read_csv(new_file)

# Combine the excel and pdf dataframes vertically into one dataframe
data = pd.concat([data1, data2], axis=0)

# Check if the 'Loan_Status_id' column exists
if 'Loan_Status_id' not in data.columns:
    """ Create the 'Loan_Status_id' column which has a 1 if the 'Loan_Status'
    column has a 'Y' and a 0 if the 'Loan_Status' column has an 'N' """

    data['Loan_Status_id'] = data['Loan_Status'].map({'Y': 1, 'N': 0})

# Create a variable  called 'predict' and assign the 'Loan_Status_id' to it.
# This is because we want to test and predict it
predict = "Loan_Status_id"

# Select all the data we need for the model
data = data[["Gender", "Married", "Dependents","Graduate", "Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History","Property_Area","Loan_Status_id"]]
data = shuffle(data) # shuffle the data

""" Create the x and y arrays where the x arrays will be the predictor
variables and the y arrays the predicted variables """
x = np.array(data.drop([predict], 1))
y =np.array(data[predict])

""" Split the data into train and test propotions depending on the
Model_Test_Size selected by the user """
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=Model_Test_Size)


"""Train the model by the number of times inputed by the user in the
Model_Iterations entry and store the model with the best accuracy
in the 'loanmodel.pickle' file """
best = 0
for _ in range(Model_Iterations):
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=Model_Test_Size)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)
    acc = linear.score(x_test, y_test)

    if acc > best:
        best = acc
        with open("loanmodel.pickle", "wb") as f:
            pickle.dump(linear, f)

"""Load the model and use it to predict the "Loan_Status_id" of the
potential loan client whose information has been entered in the form"""
pickle_in = open("loanmodel.pickle", "rb")
linear = pickle.load(pickle_in)


predicted= linear.predict(clientData)

"""Round off the predicted value to the nearest integer so that we can
know whether the predicted value is a '0' for 'N' or a '1' for 'Y' """
predictedinteger = round(predicted[0])

# Round off the model accuracy to two decimal places
ModelAccuracy = round(best,2)

""" If the predicted value is a '1' print out that the potential loan
client can be given the loan, else print out they cannot be given
the loan. The model accuracy should also be printed"""

if predictedinteger == 1:
  print("Client ", Loan_ID, " should be given the loan")
  print("Model accuracy: ", ModelAccuracy)
else:
  print("Client ", Loan_ID, " should not be given the loan")
  print("Model accuracy: ", ModelAccuracy)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-66-575467874526>:128: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(data.drop([predict], 1))


Client  11123  should be given the loan
Model accuracy:  0.55


The insight I got from running the machine learning model is that it becomes more accurate when you increase the Model_Iterations and when you reduce the Model_Test_Size. The model becomes more accurate when you train it on a larger data set than the one you use to test it.